In [3]:
import pandas as pd
import datetime
import numpy as np
import boto3
import logging
from io import StringIO

In [4]:
def download_csv_from_s3(aws_profile, bucket_name, file_keys):
    session = boto3.Session(profile_name=aws_profile)
    s3_client = session.client('s3')
    data_frames = {}

    for file_key in file_keys:
        obj = s3_client.get_object(Bucket=bucket_name, Key=file_key)
        data = obj['Body'].read().decode('utf-8')
        sep = ',' if file_key.endswith('.csv') else ';'
        data_frames[file_key] = pd.read_csv(StringIO(data), sep=sep)
    
    return data_frames

aws_profile = 'agrovid_profile'
bucket_name = 'dev-load-files-985d1fbe-9777-9d3b-1b73-de0fd6e4cb3d'
file_keys = ['er_simulado/modulo_ingresos/2024-07-24/ingreso.csv', 'er_simulado/modulo_ingresos/2024-07-24/EXCEDENTE.csv', 
             'er_simulado/modulo_ingresos/2024-07-24/platano.csv']

data_frames = download_csv_from_s3(aws_profile, bucket_name, file_keys)

# Acceder a los DataFrames descargados
surplus = data_frames['er_simulado/modulo_ingresos/2024-07-24/EXCEDENTE.csv']
revenue_file = data_frames['er_simulado/modulo_ingresos/2024-07-24/ingreso.csv']



## Importar los archivos

In [16]:
volumen = pd.read_csv('volumen.csv', sep = ';')
volumen.head()

,Unnamed: 0,CONCEPTO,FINCA,SEMANA 1,SEMANA 2,SEMANA 3,SEMANA 4,SEMANA 5,SEMANA 6,SEMANA 7,...,SEMANA 43,SEMANA 44,SEMANA 45,SEMANA 46,SEMANA 47,SEMANA 48,SEMANA 49,SEMANA 50,SEMANA 51,SEMANA 52
0,0,CAJAS,ENANO,7969.0,8319.0,7813.0,7359.0,7989.0,5479.0,6037.0,...,9164.0,9406.0,10244.0,9851.0,9751.0,9302.0,10343.0,10606.0,10349.0,9936.0
1,1,CAJAS,EVA,4098.0,6617.0,7238.0,7600.0,6414.0,4585.0,4992.0,...,7407.0,7045.0,7287.0,6578.0,6586.0,6073.0,6218.0,6091.0,5903.0,5791.0
2,2,CAJAS,SAMI,4560.0,8771.0,9549.0,9418.0,8004.0,6011.0,5031.0,...,8233.0,8250.0,8530.0,7911.0,7875.0,7546.0,8216.0,7893.0,7937.0,7645.0
3,3,CAJAS,VEGA,3625.0,3448.0,4037.0,2996.0,3455.0,2949.0,3097.0,...,4869.0,4584.0,4547.0,3897.0,3637.0,3361.0,3460.0,3426.0,3303.0,3147.0
4,4,CAJAS,FEDERICA,5265.0,5243.0,3467.0,3098.0,3646.0,4033.0,4654.0,...,10303.0,9370.0,9802.0,8779.0,8725.0,8484.0,9021.0,8505.0,7837.0,7814.0


In [18]:
corte = pd.read_csv('c&e.csv', sep = ',')
corte.head()

,ID,FINCA,MAESTRA,PROMEDIADO,PRESTACIONES,FACTOR
0,100001,ENANO,REG SUR,G04,0.3887,1.0
1,100001,EVA,REG SUR,G04,0.3887,1.0
2,100001,SAMI,REG SUR,G04,0.3887,1.0
3,100001,VEGA,REG SUR,G04,0.3887,1.0
4,100001,FEDERICA,REG SUR,G04,0.3887,1.0


## Corte y empaque

In [19]:
data_join = volumen[volumen['CONCEPTO']].join(corte, on = 'FINCA', how = 'inner')

ValueError: You are trying to merge on object and int64 columns for key 'FINCA'. If you wish to proceed you should use pd.concat